**Outline:**

1. [Libraries and settings](#1-bullet)
2. [Nested CV](#2-bullet)

## 1. [Libraries and settings](#1-bullet)

In [ ]:
from Data import *

In [41]:
import sys
import os

# Get the current working directory and the mounted drive path
# If your file is in a subdirectory of the mounted drive, specify the full path
drive_path = '/content/drive/MyDrive/VietFinanceLab-Analyzing-Market-Trends-and-Forecasting-Future-Price/SVR' # <--- REPLACE WITH YOUR ACTUAL PATH TO THE DIRECTORY CONTAINING DataPreprocessing.py

# Add the directory to the system path
if drive_path not in sys.path:
    sys.path.append(drive_path)
    print(f"Added {drive_path} to sys.path")

# Now attempt to import the module
try:
    from DataPre import *
    print("DataPreprocessing module imported successfully!")
except ModuleNotFoundError:
    print(f"Error: Module 'DataPreprocessing' not found in {sys.path}")
    print("Please verify the path to the DataPreprocessing.py file and update drive_path.")


from google.colab import drive
drive.mount('/content/drive')

DataPreprocessing module imported successfully!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
os.listdir(drive_path)


['SVR - Day forward chaining.ipynb',
 '.DS_Store',
 '.ipynb_checkpoints',
 'DataPre.py',
 'DataPreprocessing.ipynb',
 'SVR-Model.ipynb',
 '__pycache__']

## 2. [Nested CV - Day Forward Chaining](#2-bullet)

We try to use nested CV for time series to try to tune hyperparameters. After getting best hyperparameters, we retrain model and then evaluate

### Create dataset

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 30
x_for_train, y_for_train = create_dataset(scaled_data, time_step)

x_for_train = x_for_train.reshape(x_for_train.shape[0],x_for_train.shape[1])
y_for_train = y_for_train.reshape(y_for_train.shape[0],1)

### Day forward chaining

In [ ]:
def performSVR(X_train, Y_train, X_test, Y_test, parameters):

    # Create and train the SVR model
    svr_model = SVR(C = parameters['C'], kernel = parameters['kernel'], gamma = parameters['gamma'])
    svr_model.fit(X_train, Y_train)

    # Make predictions on the test set
    Y_pred_train = svr_model.predict(X_train)
    Y_pred_test = svr_model.predict(X_test)

    # Calculate accuracy
    accuracy_train = r2_score(Y_train, Y_pred_train)
    accuracy_valid = r2_score(Y_test, Y_pred_test)

    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_valid = mean_squared_error(Y_test, Y_pred_test)

    return [accuracy_train, accuracy_valid, mse_train, mse_valid]

In [ ]:
from sklearn.model_selection import ParameterGrid

def performDayForwardChaining(X_train, y_train, number_folds):
    # Định nghĩa siêu tham số để tinh chỉnh
    param_grid = {
        'C': [0.01, 0.1, 1, 100],
        'kernel': ['linear', 'rbf', 'sigmoid'],
        'gamma' : [0.01, 0.001, 1]
    }

    # Chuyển đổi siêu tham số thành danh sách các tham số có thể
    param_list = list(ParameterGrid(param_grid))

    print('Size train set: ', X_train.shape)

    # k is the size of each fold. It is computed dividing the number of
    # rows in X_train by number_folds. This number is floored and coerced to int
    k = int(np.floor(float(X_train.shape[0]) / number_folds))
    print('Size of each fold: ', k)

    results_valid = []
    results_test = []
    # loop from the first 2 folds to the total number of folds
    for i in range(3, number_folds + 1):
        print('')

        split = float(i-1)/i

        print('Splitting the first ' + str(i) + ' chunks at ' + str(i-1) + '/' + str(i))

        X = X_train[:(k*i),:]
        y = y_train[:(k*i),:]
        print('Size of train + valid + test: ', X.shape)

        index = int(np.floor(X.shape[0] * split))
        print(index)

        # folds used to train the model
        X_train_validFolds = X[:index,:]
        y_train_validFolds = y[:index,:]

        index_valid = int((i-2)*index/(i-1))

        X_trainFolds = X_train_validFolds[:index_valid,:]
        y_trainFolds = y_train_validFolds[:index_valid,:]

        X_validFolds = X_train_validFolds[index_valid:,:]
        y_validFolds = y_train_validFolds[index_valid:,:]

        # fold used to test the model
        X_testFolds = X[(index + 1):,:]
        y_testFolds = y[(index + 1):,:]

        for j,params in enumerate(param_list):
            print(f"Evaluating parameters {j+1}: {params}")
            accuracies_valid = performSVR(X_trainFolds, y_trainFolds, X_validFolds, y_validFolds, params)

            results_valid.append({
            'parameters': params,
            'accuracy_train': accuracies_valid[0],
            'accuracy_valid': accuracies_valid[1],
            'mse_train': accuracies_valid[2],
            'mse_valid': accuracies_valid[3]
            })

        for result in results_valid:
            print("Parameters: ", result['parameters'])
            print("Train Accuracy: ", result['accuracy_train'])
            print("Valid Accuracy: ", result['accuracy_valid'])
            print("Train MSE: ", result['mse_train'])
            print("Valid MSE: ", result['mse_valid'])
            print("------------------------")

        # Sắp xếp danh sách kết quả theo trường 'accuracy_valid' tăng dần
        sorted_results_valid = sorted(results_valid, key=lambda x: x['mse_valid'], reverse=False)

        # Lấy bộ tham số đầu tiên từ danh sách đã sắp xếp
        best_parameters = sorted_results_valid[0]['parameters']
        print(best_parameters)

        accuracies_test = performSVR(X_train_validFolds, y_train_validFolds, X_testFolds, y_testFolds, best_parameters)

        results_test.append({
            'parameters': best_parameters,
            'accuracy_train': accuracies_test[0],
            'accuracy_test': accuracies_test[1],
            'mse_train': accuracies_test[2],
            'mse_test': accuracies_test[3]
            })

    acc_train_model = 0
    acc_test_model = 0
    mse_train_model = 0
    mse_test_model = 0
    cnt = 0
    for result in results_test:
        acc_train_model += result['accuracy_train']
        acc_test_model += result['accuracy_test']
        mse_train_model += result['mse_train']
        mse_test_model += result['mse_test']
        cnt += 1

    acc_train_model = acc_train_model/cnt
    acc_test_model = acc_test_model/cnt
    mse_train_model = mse_train_model/cnt
    mse_test_model = mse_test_model/cnt
    # the function returns the mean of the accuracy on the n-1 folds
    return acc_train_model, acc_test_model, mse_train_model, mse_test_model

In [ ]:
accuracy_train_model, accuracy_test_model, mse_train_model, mse_test_model = performDayForwardChaining(x_for_train, y_for_train, number_folds=5)

print("Train Accuracy Model: ", accuracy_train_model)
print("Test Accuracy Model: ", accuracy_test_model)
print("Train MSE Model: ", mse_train_model)
print("Test MSE Model: ", mse_test_model)
print("------------------------")


Size train set:  (4475, 30)
Size of each fold:  895

Splitting the first 3 chunks at 2/3
Size of train + valid + test:  (2685, 30)
1790
Evaluating parameters 1: {'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}
Evaluating parameters 2: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Evaluating parameters 3: {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
Evaluating parameters 4: {'C': 0.01, 'gamma': 0.001, 'kernel': 'linear'}
Evaluating parameters 5: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
Evaluating parameters 6: {'C': 0.01, 'gamma': 0.001, 'kernel': 'sigmoid'}
Evaluating parameters 7: {'C': 0.01, 'gamma': 1, 'kernel': 'linear'}
Evaluating parameters 8: {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
Evaluating parameters 9: {'C': 0.01, 'gamma': 1, 'kernel': 'sigmoid'}
Evaluating parameters 10: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}
Evaluating parameters 11: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Evaluating parameters 12: {'C': 0.1, 'gamma': 0.01, 'kernel': 'sigmoid'}
Evaluating par